In [1]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tqdm import tqdm

class FocalLoss(nn.Module):
    def __init__(self, gamma=2, alpha=None):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha

    def forward(self, input, target):
        ce_loss = F.cross_entropy(input, target, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma) * ce_loss
        if self.alpha is not None:
            focal_loss = self.alpha * focal_loss
        return focal_loss.mean()

root_dir = r'/Users/roy/Downloads/archive/'  # Ruta del directorio de imágenes

N_IDC = []
P_IDC = []

for dir_name in tqdm(os.listdir(root_dir)):
    dir_path = os.path.join(root_dir, dir_name)
    if os.path.isdir(dir_path):
        negative_dir_path = os.path.join(dir_path, '0')
        positive_dir_path = os.path.join(dir_path, '1')
        if os.path.isdir(negative_dir_path) and os.path.isdir(positive_dir_path):
            negative_image_paths = [
                os.path.join(negative_dir_path, image_name)
                for image_name in os.listdir(negative_dir_path)
                if image_name.endswith('.png')
            ]
            positive_image_paths = [
                os.path.join(positive_dir_path, image_name)
                for image_name in os.listdir(positive_dir_path)
                if image_name.endswith('.png')
            ]
            N_IDC.extend(negative_image_paths)
            P_IDC.extend(positive_image_paths)

total_images = 5000  # Cambiado a 5000 para equilibrar las clases (2500 benignos y 2500 malignos)

n_img_arr = np.zeros(shape=(total_images, 50, 50, 3), dtype=np.float32)
p_img_arr = np.zeros(shape=(total_images, 50, 50, 3), dtype=np.float32)
label_n = np.zeros(total_images)
label_p = np.ones(total_images)

for i, img in tqdm(enumerate(N_IDC[:total_images])):
    n_img = cv2.imread(img, cv2.IMREAD_COLOR)
    n_img_size = cv2.resize(n_img, (50, 50), interpolation=cv2.INTER_LINEAR)
    n_img_arr[i] = n_img_size

for i, img in tqdm(enumerate(P_IDC[:total_images])):
    p_img = cv2.imread(img, cv2.IMREAD_COLOR)
    p_img_size = cv2.resize(p_img, (50, 50), interpolation=cv2.INTER_LINEAR)
    p_img_arr[i] = p_img_size

X = np.concatenate((p_img_arr, n_img_arr), axis=0)
y = np.concatenate((label_p, label_n), axis=0)
X, y = shuffle(X, y, random_state=0)

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)


100%|██████████| 281/281 [00:02<00:00, 120.81it/s]
5000it [00:02, 2441.88it/s]
5000it [00:02, 2417.11it/s]


In [2]:
class CNN(nn.Module):
    def __init__(self, output_size=2):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=4, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        self.dropout1 = nn.Dropout2d(p=0.3)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(64)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(12 * 12 * 64, 64)
        self.bn5 = nn.BatchNorm1d(64)
        self.fc2 = nn.Linear(64, 64)
        self.dropout2 = nn.Dropout(p=0.3)
        self.fc3 = nn.Linear(64, 24)
        self.fc4 = nn.Linear(24, output_size)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.bn1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.bn2(x)
        x = self.maxpool1(x)
        x = self.dropout1(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = self.bn3(x)
        x = self.conv4(x)
        x = F.relu(x)
        x = self.bn4(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.bn5(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        x = F.relu(x)
        x = self.fc4(x)
        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN().to(device)
criterion = FocalLoss(gamma=2, alpha=None)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [3]:

def train(model, device, train_loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_loss = running_loss / len(train_loader)
    return train_loss

def evaluate(model, device, val_loader, criterion):
    model.eval()
    val_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            val_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    val_loss /= len(val_loader)
    accuracy = 100.0 * correct / len(val_loader.dataset)
    return val_loss, accuracy

In [4]:
batch_size = 32  # Define the batch size
train_dataset = TensorDataset(torch.from_numpy(X_train).float().permute(0, 3, 1, 2), torch.from_numpy(y_train).long())
val_dataset = TensorDataset(torch.from_numpy(X_val).float().permute(0, 3, 1, 2), torch.from_numpy(y_val).long())
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

num_epochs = 40
best_val_loss = float('inf')

for epoch in range(num_epochs):
    model.train()
    train_loss = train(model, device, train_loader, optimizer, criterion)
    model.eval()
    val_loss, val_accuracy = evaluate(model, device, val_loader, criterion)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pt')

    print('Epoch: {:02d}, Train Loss: {:.4f}, Val Loss: {:.4f}, Val Accuracy: {:.2f}%'.format(
        epoch, train_loss, val_loss, val_accuracy))

# Load the best model
model.load_state_dict(torch.load('best_model.pt'))

Epoch: 00, Train Loss: 0.0913, Val Loss: 0.1486, Val Accuracy: 81.28%
Epoch: 01, Train Loss: 0.0740, Val Loss: 0.1635, Val Accuracy: 69.17%
Epoch: 02, Train Loss: 0.0746, Val Loss: 0.0891, Val Accuracy: 86.17%
Epoch: 03, Train Loss: 0.0636, Val Loss: 0.0630, Val Accuracy: 90.22%
Epoch: 04, Train Loss: 0.0670, Val Loss: 0.6854, Val Accuracy: 62.39%
Epoch: 05, Train Loss: 0.0645, Val Loss: 0.0781, Val Accuracy: 86.78%
Epoch: 06, Train Loss: 0.0591, Val Loss: 0.4326, Val Accuracy: 50.33%
Epoch: 07, Train Loss: 0.0584, Val Loss: 0.0691, Val Accuracy: 87.72%
Epoch: 08, Train Loss: 0.0514, Val Loss: 0.4015, Val Accuracy: 64.94%
Epoch: 09, Train Loss: 0.0499, Val Loss: 0.0593, Val Accuracy: 90.72%
Epoch: 10, Train Loss: 0.0475, Val Loss: 0.1264, Val Accuracy: 81.06%
Epoch: 11, Train Loss: 0.0486, Val Loss: 0.1976, Val Accuracy: 76.06%
Epoch: 12, Train Loss: 0.0446, Val Loss: 0.2016, Val Accuracy: 77.44%
Epoch: 13, Train Loss: 0.0458, Val Loss: 1.3397, Val Accuracy: 65.78%
Epoch: 14, Train Los

<All keys matched successfully>

In [7]:
import matplotlib.pyplot as plt

def plot_loss(train_loss, val_loss):
    epochs = range(1, len(train_loss) + 1)

    plt.plot(epochs, train_loss, label='Train Loss')
    plt.plot(epochs, val_loss, label='Val Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.show()



In [6]:
plot_losses(train_loss, val_loss)

TypeError: object of type 'float' has no len()